In [107]:
from text2vec import SentenceModel
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import numpy as np
m = SentenceModel("shibing624/text2vec-base-chinese-paraphrase", max_seq_length=512, device="cuda")

2024-03-19 16:46:00.324 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cuda


In [108]:
df = pd.read_csv("train.csv")
df = df[["content", "topic"]]

In [109]:
cls_emb_center = {}
for i in tqdm(range(24)):
    t = df[df['topic'] == i]
    t_emb = [m.encode(i) for i in t['content']]
    t_emb_mean = np.mean(t_emb, axis=0)
    cls_emb_center[i] = t_emb_mean

100%|██████████| 24/24 [04:24<00:00, 11.01s/it]


In [117]:
import pickle
with open("cls_emb_center_512.pkl", "wb") as f:
    pickle.dump(cls_emb_center, f)

In [118]:
test = pd.read_csv("test.csv")
test = test[["content", "topic"]]

In [124]:
from sklearn.metrics.pairwise import cosine_similarity

test_emb = [m.encode(i) for i in tqdm(test['content'])]

preds = []
max_sim = []
for i in test_emb:
    sim = [cosine_similarity(i.reshape(1, -1), cls_emb_center[j].reshape(1, -1))[0][0] for j in range(24)]
    max_sim.append(max(sim))
    pred = np.argmax(sim)
    preds.append(pred)

test['pred'] = preds
test["max_sim"] = max_sim

100%|██████████| 3405/3405 [02:12<00:00, 25.68it/s]


In [125]:
test

,content,topic,pred,max_sim
0,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,1,4,0.948044
1,当地时间11月16日，习近平主席向亚太经合组织（APEC）工商领导人峰会发表书面演讲，强调我...,1,1,0.945361
2,亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。国家主席习近平出席会议并发...,1,1,0.961683
3,习近平出席亚太经合组织领导人同东道主嘉宾非正式对话会暨工作午宴央视网2023年11月17日 ...,1,1,0.930212
4,中国国家主席习近平将于11月14日应美国总统拜登邀请赴美举行中美领导人会晤，同时应邀出席亚太...,1,4,0.972955
...,...,...,...,...
3400,从头到尾来讲一遍，咱们按照惯例，先谈谈缅甸北部的那场风波。这场风波的源头是藏匿在缅甸北部的一...,23,23,0.908576
3401,缅北全线激战，敏昂莱誓言反击，冲突可能扩大化，中缅边境乱不得这几天，缅甸内战的爆发，引起了外...,23,23,0.933027
3402,近日，缅甸内战再次卷土重来，引起了国际社会广泛的关注。然而，这次的冲突异常纷乱，各种消息传来...,23,23,0.944409
3403,缅北战火越烧越大，中方做两手准备，高层直接去了缅甸首都清酒半栩换种角度分析故事，带你了解更有...,23,23,0.951230


In [126]:
diff = test[test['topic'] != test['pred']]

In [127]:
diff

,content,topic,pred,max_sim
0,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,1,4,0.948044
4,中国国家主席习近平将于11月14日应美国总统拜登邀请赴美举行中美领导人会晤，同时应邀出席亚太...,1,4,0.972955
7,APEC峰会前，我公使提出四点核心利益，中美是否会晤，得看美表现“一带一路”国际高峰论坛取得...,1,4,0.946068
9,美国态度大变，宣布邀请俄罗斯参加明年APEC会议，普京恐不会出席2023年，美国将担任亚太经...,1,3,0.895437
32,中国国家主席习近平将于11月14日应美国总统拜登邀请赴美举行中美领导人会晤，同时应邀出席亚太...,1,4,0.971747
...,...,...,...,...
3127,神舟十七号发射成功：续写飞天梦想，建设航天强国。北京时间10月26日11时13分59秒，搭载...,21,20,0.985819
3201,新华网10月18日上午，国家主席习近平夫人彭丽媛邀请出席第三届“一带一路”国际合作高峰论坛的...,22,4,0.847199
3308,"缅甸历史史前历史前11,000年–前200年骠国前200年–1050年孟王国825年?–10...",23,0,0.894279
3328,5月2日，中国国务委员兼外交部长秦刚开始访问缅甸。外媒注意到这是中方高层在缅甸政权更迭后首次...,23,4,0.881124


In [123]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(test['topic'], test['pred'])
precision = precision_score(test['topic'], test['pred'], average='weighted')
recall = recall_score(test['topic'], test['pred'], average='weighted')
f1 = f1_score(test['topic'], test['pred'], average='weighted')
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}')

Accuracy: 0.9262848751835536
Precision: 0.9341271265254506
Recall: 0.9262848751835536
F1: 0.9266618882472175
